In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_colwidth', None)
import os
home_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
from tqdm import tqdm



index_selected_list = ['上证50','上证150','沪深300','中证500','中证1000','科创50','创业板50','创业300']
ignored_list = ['上证180','上证380','深证100','深证200','深证700']
concepts_list = pd.read_csv(home_path + '/static/concepts_checklist.csv',encoding="utf-8")
industry_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['行业','产品','业务','行业+']) & concepts_list['active']==1 ]['概念名称'].values.tolist()
events_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['事件','改革','政策']) & concepts_list['active']==1]['概念名称'].values.tolist()
master_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['知名企业','国企体系']) & concepts_list['active']==1]['概念名称'].values.tolist()
exclude_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['交易'])]['概念名称'].values.tolist()

wc_result = pd.DataFrame()

for i in tqdm(range(1,2)):
    print('page: '+ str(i))
    sr = wc.search('2023年预测净利润 总股本 行业 所属指数类 公司亮点',page =i).fillna('')
    sr['沪深指数'] = ''
    sr['行业概念'] = ''
    sr['事件概念'] = ''
    sr['体系概念'] = ''

    for row, l in sr.iterrows():
        # sr['主营产品名称'][row] = ','.join(l['主营产品名称'].split('||'))
        sr['沪深指数'][row] = ','.join(set(l['所属指数类'].split(';')).intersection(index_selected_list))
        sr['所属概念'][row] = ','.join(set(l['所属概念'].split(';')).difference(exclude_concepts_list))
        # sr['行业概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(industry_concepts_list)).replace('金属','')
        # sr['事件概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(events_concepts_list))
        # sr['体系概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(master_concepts_list))

    sr['所属概念'] =   sr['所属概念'].str.replace('概念股','').str.replace('概念','')
#     sr['流动市值'] = (pd.to_numeric(sr['a股市值(不含限售股)'])/100000000).round(1).fillna('')    
    

In [2]:
import requests

url = 'http://www.iwencai.com/gateway/urp/v7/landing/getDataList'

query_string = '所属概念 公司亮点 所属同花顺行业 所属指数类'
page = 2

condition = []
condition_1 = {
        'score': 0.0,
        'chunkedResult': '_&_'.join(query_string.split(' ')),
        'opName': 'and',
        'opProperty': '',
        'sonSize': 3,
        'relatedSize': '0',
        'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
        'source': 'text2sql'
        }
condition.append(condition_1)

for i in query_string.split(' '):
#     i = i.replace('指数','指数类')
    dic = {
        'dateText': '',
        'indexName': i,
        'indexProperties': [],
        'child_node': i,
        'ci': True,
        'source': 'text2sql',
        'type': 'index',
        'indexPropertiesMap': {},
        'reportType': 'null',
        'ciChunk': i,
        'createBy': 'preCache',
        'uiText': i,
        'valueType': '_' + i,
        'domain': 'abs_股票领域',
        'sonSize': 0,
        'dateList': []
        }
    condition.append(dic)
condition = str(condition).replace("'", '"').replace(' ','')


payload = {
    "question": query_string,
    "page": page,
    "perpage": 100,
    "source": "Ths_iwencai_Xuangu",
    'urp_sort_way': 'desc',
    'urp_sort_index': '最新涨跌幅',
    'addheaderindexes': '',
    'codelist': '',
    'indexnamelimit': '',
    'ret': 'json_all',
    'urp_use_sort': '1',
    'user_id': '570662655',
    'uuids[0]': '24087',
    'query_type': 'stock',
    'comp_id': '6734520',
    'business_cat': 'soniu',
    'uuid': '24087',
#    以下为每次变化
#     'date_range[0]': '20230610',
#     'iwc_token': '0ac9529816863435305771116',
#     'sessionid': '9f98f2743c41f01ae1e20f42ce38d2a1',
#     'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
    'condition': str(condition).replace("'", '"').replace(' ','')
}

cookies = {
    "WafStatus":'0',
    "ta_random_userid":'anro5tt7uu',
    "cid":'e77a8079986b06e6a9a46e614cf7eee21659209895',
    "ComputerID":'e77a8079986b06e6a9a46e614cf7eee21659209895',
    "other_uid":'Ths_iwencai_Xuangu_x25492z942nd2ag2aqbfqwh7m852m7vi',
    "ttype":'WEB',
    "wencai_pc_version":'1',
    "user_status":'0',
    "user":'MDpteF81NzA2NjI2NTU6Ok5vbmU6NTAwOjU4MDY2MjY1NTo3LDExMTExMTExMTExLDQwOzQ0LDExLDQwOzYsMSw0MDs1LDEsNDA6MTY6Ojo1NzA2NjI2NTU6MTY4NjMyMTg1NTo6OjE2MTQ4OTgzMjA6NjA0ODAwOjA6MTk0Yzg1NGMxMzdlYzY1ZmY1NTcxNjIzMWU2YzA2MGZkOmRlZmF1bHRfNDow',
    "userid":'570662655',
    "u_name":'mx_570662655',
    "escapename":'mx_570662655',
    "ticket":'106f915b0a9985030a58c2151f69d9b8',
    "utk":'86be216097cbd990b074d0a691f39d39',
#     'THSSESSID':'210fbd845858470fd3dcc8c227',
#     "PHPSESSID":'i1jarg6eiikd16j6ajcjuqhb7j1a88je',
#     "v":'A1hilGtTiE5eR6Q8zbqmwNg1KY3vQa3eHoyRWZLcpY9GCvSzOlGMW261YLTh',
}


headers = {
    'Accept':'application/json, text/plain, */*',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,en-US;q=0.7,ja;q=0.6,zh-TW;q=0.5,nl;q=0.4',
    'Cache-Control':'no-cache',
    'Connection':'keep-alive',
    'Content-Length':'2221',
    'Content-Type':'application/x-www-form-urlencoded',
#     'Hexin-V':'A1hilGtTiE5eR6Q8zbqmwNg1KY3vQa3eHoyRWZLcpY9GCvSzOlGMW261YLTh',
    'Host':'www.iwencai.com',
    'Origin':'http://www.iwencai.com',
    'Pragma':'no-cache',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}



r = requests.post(url, data = payload, cookies = cookies, headers = headers)

import json
import pandas as pd
df = pd.DataFrame(r.json()['answer']['components'][0]['data']['datas'])
df

,股票代码,股票简称,最新价,最新涨跌幅,所属概念,公司亮点,所属同花顺行业,所属指数类,market_code,code
0,002271.SZ,东方雨虹,28.68,7.982,参股保险;新材料概念;空中巴士;电子商务;参股民营银行;高铁;海底隧道;铁路基建;融资融券;...,建筑防水材料行业龙头,建筑材料-建筑材料-其他建材,300 ESG;300精明;300精明HKD;300精明USD;中创100;中创500;中小...,33,002271
1,688339.SH,亿华通,86.76,7.910,燃料电池;融资融券;转融券标的;专精特新;氢能源;沪股通,中国氢能产业先行者,电力设备-电力设备-电池,A股指数;上证指数;上证流通;中证1000;中证1000USD;中证A股;中证全指;中证流通...,17,688339
2,002182.SZ,云海金属,21.86,7.844,液态金属;3D打印;新材料概念;轻型合金;金属镁;富士康概念;有色铝;小金属概念;融资融券;...,国内最大专业化镁合金生产商,有色金属-小金属-其他小金属,中创400;中创500;中小300;中小创新;中小基础;中小综指;中证1000;中证1000...,33,002182
3,300317.SZ,珈伟新能,5.66,7.810,人民币贬值受益;节能照明;光伏概念;OLED;柔性屏;储能;绿色电力;地方国企改革;国企改革,阜阳颍泉区国资委旗下，最早研究太阳能与LED综合应用技术并实现规模化生产，建成全球最大的斜单...,公用事业-电力-新能源发电,中证A股;中证全指;中证流通;创业基础;创业板综;制造指数;同花顺全A;新指数;深证A指;深...,33,300317
4,300875.SZ,捷强装备,42.50,7.459,军工;融资融券;转融券标的;核电;高端装备;体外诊断;专精特新;核污染防治;机器人概念;高送转预期,荣获国家科学技术进步奖二等奖、中国人民解放军科技进步奖二等奖,国防军工-国防军工-地面兵装,专用指数;中证A股;中证全指;中证流通;创业板综;制造指数;同花顺全A;新指数;深证A指;深证综指,33,300875
...,...,...,...,...,...,...,...,...,...,...
95,605299.SH,舒华体育,12.61,5.966,体育产业;健康中国;奥运会;华为概念;冬奥会;杭州亚运会,北京冬奥会官方健身器材供应商，主营健身器材和展示架,轻工制造-家用轻工-文娱用品,A股指数;上证指数;上证流通;中型综指;中证A股;中证全指;中证流通;同花顺全A;工业指数;新综指,17,605299
96,600266.SH,城建发展,5.34,5.952,参股新三板;棚户区改造;融资融券;转融券标的;创投;富时罗素概念股;环球主题公园;北交所概念...,具有房地产开发一级资质，地产项目主要在北京,房地产-房地产开发-住宅开发,A股指数;上证指数;上证流通;中证1000;中证1000USD;中证A股;中证全指;中证流通...,17,600266
97,600322.SH,天房发展,1.79,5.917,京津冀一体化;滨海新区;环渤海;棚户区改造;天津自贸区;融资融券;转融券标的;地方国企改革;...,聚焦天津地区的房地产开发商,房地产-房地产开发-住宅开发,A股指数;上证指数;上证流通;中证A股;中证全指;中证流通;同花顺全A;地产指数;新综指,17,600322
98,003021.SZ,兆威机电,80.55,5.917,5G;华为概念;机器人概念;新能源汽车;小米概念;华为汽车;深股通;比亚迪概念;融资融券;转...,国内少有、全球领先的微型传动系统制造商，曾获颁国家科学进步奖二等奖,电力设备-电力设备-电机,中小300;中小创新;中小基础;中小综指;中证1000;中证1000USD;中证A股;中证全...,33,003021


In [165]:
a = '[{"score":0.0,"chunkedResult":"所属概念_&_所属指数_&_公司亮点","opName":"and","opProperty":"","sonSize":3,"relatedSize":"0","logid":"9f98f2743c41f01ae1e20f42ce38d2a1","source":"text2sql"},{"dateText":"","indexName":"所属概念","indexProperties":[],"child_node":"所属概念","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属概念","createBy":"preCache","uiText":"所属概念","valueType":"_所属概念","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"所属指数类","indexProperties":[],"child_node":"所属指数类","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属指数","createBy":"preCache","uiText":"所属指数类","valueType":"_所属指数类","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"公司亮点","indexProperties":[],"child_node":"公司亮点","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"公司亮点","createBy":"preCache","uiText":"公司亮点","valueType":"_公司亮点","domain":"abs_股票领域","sonSize":0,"dateList":[]}]'


In [131]:
query_string = '所属概念 所属指数 公司亮点 所属行业'
'_&_'.join(query_string.split(' '))

'所属概念_&_所属指数_&_公司亮点_&_所属行业'

In [167]:
a

'[{"score":0.0,"chunkedResult":"所属概念_&_所属指数_&_公司亮点","opName":"and","opProperty":"","sonSize":3,"relatedSize":"0","logid":"9f98f2743c41f01ae1e20f42ce38d2a1","source":"text2sql"},{"dateText":"","indexName":"所属概念","indexProperties":[],"child_node":"所属概念","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属概念","createBy":"preCache","uiText":"所属概念","valueType":"_所属概念","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"所属指数类","indexProperties":[],"child_node":"所属指数类","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属指数","createBy":"preCache","uiText":"所属指数类","valueType":"_所属指数类","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"公司亮点","indexProperties":[],"child_node":"公司亮点","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"公司亮点","createBy":"preCache","uiText":"公司亮点","valueType":"_公司亮点","domain"

In [170]:
str(condition).replace("'", '"').replace(' ','')

'[{"score":0.0,"chunkedResult":"所属概念_&_所属指数_&_公司亮点","opName":"and","opProperty":"","sonSize":3,"relatedSize":"0","logid":"9f98f2743c41f01ae1e20f42ce38d2a1","source":"text2sql"},{"dateText":"","indexName":"所属概念","indexProperties":[],"child_node":"所属概念","ci":True,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属概念","createBy":"preCache","uiText":"所属概念","valueType":"_所属概念","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"所属指数","indexProperties":[],"child_node":"所属指数","ci":True,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属指数","createBy":"preCache","uiText":"所属指数","valueType":"_所属指数类","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"公司亮点","indexProperties":[],"child_node":"公司亮点","ci":True,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"公司亮点","createBy":"preCache","uiText":"公司亮点","valueType":"_公司亮点","domain":"a

In [ ]:



{'score': 0.0,
  'chunkedResult': '_&_'.join(query_string.split(' ')),
  'opName': 'and',
  'opProperty': '',
  'sonSize': 3,
  'relatedSize': '0',
#   'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
  'source': 'text2sql'}


{'dateText': '',
  'indexName': '所属概念',
  'indexProperties': [],
  'child_node': '所属概念',
  'ci': True,
  'source': 'text2sql',
  'type': 'index',
  'indexPropertiesMap': {},
  'reportType': 'null',
  'ciChunk': '所属概念',
  'createBy': 'preCache',
  'uiText': '所属概念',
  'valueType': '_所属概念',
  'domain': 'abs_股票领域',
  'sonSize': 0,
  'dateList': []},

In [128]:
json.loads(a[0])

[{'score': 0.0,
  'chunkedResult': '所属概念_&_所属指数_&_公司亮点',
  'opName': 'and',
  'opProperty': '',
  'sonSize': 3,
  'relatedSize': '0',
  'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
  'source': 'text2sql'},
 {'dateText': '',
  'indexName': '所属概念',
  'indexProperties': [],
  'child_node': '所属概念',
  'ci': True,
  'source': 'text2sql',
  'type': 'index',
  'indexPropertiesMap': {},
  'reportType': 'null',
  'ciChunk': '所属概念',
  'createBy': 'preCache',
  'uiText': '所属概念',
  'valueType': '_所属概念',
  'domain': 'abs_股票领域',
  'sonSize': 0,
  'dateList': []},
 {'dateText': '',
  'indexName': '所属指数类',
  'indexProperties': [],
  'child_node': '所属指数类',
  'ci': True,
  'source': 'text2sql',
  'type': 'index',
  'indexPropertiesMap': {},
  'reportType': 'null',
  'ciChunk': '所属指数',
  'createBy': 'preCache',
  'uiText': '所属指数类',
  'valueType': '_所属指数类',
  'domain': 'abs_股票领域',
  'sonSize': 0,
  'dateList': []},
 {'dateText': '',
  'indexName': '公司亮点',
  'indexProperties': [],
  'child_node': '公司亮点',
  